# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 20 2022 8:30AM,249008,10,0086088472,ISDIN Corporation,Released
1,Oct 20 2022 8:30AM,249008,10,0086088474,ISDIN Corporation,Released
2,Oct 20 2022 8:30AM,249008,10,0086088473,ISDIN Corporation,Released
3,Oct 20 2022 8:30AM,249008,10,0086088484,ISDIN Corporation,Released
4,Oct 20 2022 8:30AM,249008,10,0086088485,ISDIN Corporation,Released
5,Oct 20 2022 8:30AM,249008,10,0086088488,ISDIN Corporation,Released
6,Oct 20 2022 8:30AM,249008,10,0086088489,ISDIN Corporation,Released
7,Oct 20 2022 8:30AM,249008,10,0086088487,ISDIN Corporation,Released
8,Oct 20 2022 8:30AM,249008,10,0086088486,ISDIN Corporation,Released
9,Oct 20 2022 8:30AM,249008,10,0086088512,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,249043,Released,1
24,249045,Released,1
25,249046,Released,1
26,249047,Released,1
27,249048,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249043,NaN,NaN,1.0
249045,NaN,NaN,1.0
249046,NaN,NaN,1.0
249047,NaN,NaN,1.0
249048,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0.0,0.0,1.0
248946,8.0,24.0,3.0
248961,0.0,0.0,1.0
248979,0.0,0.0,2.0
248983,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0,0,1
248946,8,24,3
248961,0,0,1
248979,0,0,2
248983,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,0,0,1
1,248946,8,24,3
2,248961,0,0,1
3,248979,0,0,2
4,248983,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,,,1
1,248946,8,24,3
2,248961,,,1
3,248979,,,2
4,248983,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation
35,Oct 20 2022 8:30AM,249009,10,ISDIN Corporation
37,Oct 20 2022 8:30AM,249014,10,ISDIN Corporation
47,Oct 20 2022 8:30AM,249015,10,Methapharm-G
48,Oct 20 2022 8:30AM,249016,10,Methapharm-G
49,Oct 20 2022 8:06AM,249048,10,Emerginnova
55,Oct 20 2022 8:05AM,249047,16,TASA USA Inc.
56,Oct 20 2022 8:01AM,249046,19,"SD Head USA, LLC"
57,Oct 20 2022 8:01AM,249045,16,TASA USA Inc.
58,Oct 20 2022 7:56AM,249043,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation,,,35
1,Oct 20 2022 8:30AM,249009,10,ISDIN Corporation,,,2
2,Oct 20 2022 8:30AM,249014,10,ISDIN Corporation,,,10
3,Oct 20 2022 8:30AM,249015,10,Methapharm-G,,,1
4,Oct 20 2022 8:30AM,249016,10,Methapharm-G,,,1
5,Oct 20 2022 8:06AM,249048,10,Emerginnova,,,6
6,Oct 20 2022 8:05AM,249047,16,TASA USA Inc.,,,1
7,Oct 20 2022 8:01AM,249046,19,"SD Head USA, LLC",,,1
8,Oct 20 2022 8:01AM,249045,16,TASA USA Inc.,,,1
9,Oct 20 2022 7:56AM,249043,16,TASA USA Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation,35,,
1,Oct 20 2022 8:30AM,249009,10,ISDIN Corporation,2,,
2,Oct 20 2022 8:30AM,249014,10,ISDIN Corporation,10,,
3,Oct 20 2022 8:30AM,249015,10,Methapharm-G,1,,
4,Oct 20 2022 8:30AM,249016,10,Methapharm-G,1,,
5,Oct 20 2022 8:06AM,249048,10,Emerginnova,6,,
6,Oct 20 2022 8:05AM,249047,16,TASA USA Inc.,1,,
7,Oct 20 2022 8:01AM,249046,19,"SD Head USA, LLC",1,,
8,Oct 20 2022 8:01AM,249045,16,TASA USA Inc.,1,,
9,Oct 20 2022 7:56AM,249043,16,TASA USA Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation,35,,
1,Oct 20 2022 8:30AM,249009,10,ISDIN Corporation,2,,
2,Oct 20 2022 8:30AM,249014,10,ISDIN Corporation,10,,
3,Oct 20 2022 8:30AM,249015,10,Methapharm-G,1,,
4,Oct 20 2022 8:30AM,249016,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation,35,NaN,NaN
1,Oct 20 2022 8:30AM,249009,10,ISDIN Corporation,2,NaN,NaN
2,Oct 20 2022 8:30AM,249014,10,ISDIN Corporation,10,NaN,NaN
3,Oct 20 2022 8:30AM,249015,10,Methapharm-G,1,NaN,NaN
4,Oct 20 2022 8:30AM,249016,10,Methapharm-G,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation,35,0.0,0.0
1,Oct 20 2022 8:30AM,249009,10,ISDIN Corporation,2,0.0,0.0
2,Oct 20 2022 8:30AM,249014,10,ISDIN Corporation,10,0.0,0.0
3,Oct 20 2022 8:30AM,249015,10,Methapharm-G,1,0.0,0.0
4,Oct 20 2022 8:30AM,249016,10,Methapharm-G,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3486019,97,24.0,8.0
16,996123,4,0.0,0.0
19,1743148,46,0.0,0.0
20,248961,1,0.0,0.0
21,248896,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3486019,97,24.0,8.0
1,16,996123,4,0.0,0.0
2,19,1743148,46,0.0,0.0
3,20,248961,1,0.0,0.0
4,21,248896,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,97,24.0,8.0
1,16,4,0.0,0.0
2,19,46,0.0,0.0
3,20,1,0.0,0.0
4,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,97.0
1,16,Released,4.0
2,19,Released,46.0
3,20,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,8.0,0.0,0.0,0.0,0.0
Executing,24.0,0.0,0.0,0.0,0.0
Released,97.0,4.0,46.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,24.0,0.0,0.0,0.0,0.0
2,Released,97.0,4.0,46.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,24.0,0.0,0.0,0.0,0.0
2,Released,97.0,4.0,46.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()